### Original writer HarisMichailidis

In [22]:
with open('C:/Users/Sotiris/Desktop/project/data/auth.log') as f:
    data = f.readlines()

In [24]:
data[:10]

['Mar 27 13:06:56 ip-10-77-20-248 sshd[1291]: Server listening on 0.0.0.0 port 22.\n',
 'Mar 27 13:06:56 ip-10-77-20-248 sshd[1291]: Server listening on :: port 22.\n',
 'Mar 27 13:06:56 ip-10-77-20-248 systemd-logind[1118]: Watching system buttons on /dev/input/event0 (Power Button)\n',
 'Mar 27 13:06:56 ip-10-77-20-248 systemd-logind[1118]: Watching system buttons on /dev/input/event1 (Sleep Button)\n',
 'Mar 27 13:06:56 ip-10-77-20-248 systemd-logind[1118]: New seat seat0.\n',
 'Mar 27 13:08:09 ip-10-77-20-248 sshd[1361]: Accepted publickey for ubuntu from 85.245.107.41 port 54259 ssh2: RSA SHA256:Kl8kPGZrTiz7g4FO1hyqHdsSBBb5Fge6NWOobN03XJg\n',
 'Mar 27 13:08:09 ip-10-77-20-248 sshd[1361]: pam_unix(sshd:session): session opened for user ubuntu by (uid=0)\n',
 'Mar 27 13:08:09 ip-10-77-20-248 systemd: pam_unix(systemd-user:session): session opened for user ubuntu by (uid=0)\n',
 'Mar 27 13:08:09 ip-10-77-20-248 systemd-logind[1118]: New session 1 of user ubuntu.\n',
 'Mar 27 13:09:37

In [26]:
from pygrok import Grok

In [27]:
pattern = '%{SYSLOGTIMESTAMP:date} %{SYSLOGHOST:hostname} sshd(?:\\[%{POSINT}\\])?: (?:Failed password for invalid user|Failed password for|Invalid user|Failed publickey for) %{DATA:user} from %{IPORHOST:ip}'
grok = Grok(pattern)

In [28]:
# Modified the preprocessing so that the same results as in elasticsearch x-pack example occur

def get_fail_line_info(line):
    grok_match = grok.match(line)
    if grok_match is None:
        return
    date = grok_match['date']
    hostname = grok_match['hostname']
    usr = grok_match['user']
    ip = grok_match['ip']
    # TODO change to 2017. changed this for comparing of timestamps
    return ['2018 ' + date, hostname, usr, ip]

In [29]:
lines = [d for d in data if 'Failed' in d or 'Invalid' in d]

In [30]:
out_matrix = []
for line in lines:
    info = get_fail_line_info(line)
    if info:
        # add logline to the info
        info.append(line)
        out_matrix.append(info)

In [31]:
import pandas as pd
%matplotlib inline

In [2]:
import datetime

epoch = datetime.datetime.utcfromtimestamp(0)

def unix_time_millis(dt):
    return (dt - epoch).total_seconds() * 1000.0

pd.set_option('display.float_format', lambda x: '%.0f' % x)


df = pd.DataFrame(out_matrix,columns=['date','hostname','user','ip','logline'])

# keep date format for human view
# df['date'] = pd.to_datetime(df['date'],format='%Y %b %d %H:%M:%S')

df['date'] = pd.to_datetime(df['date'])
df.index = df['date']

df['timestamp'] = [unix_time_millis(d) for d in df.index]


df = df[['timestamp','hostname','user','ip','logline']]
df.info()

NameError: name 'pd' is not defined

In [3]:
df.head()

NameError: name 'pd' is not defined

In [34]:
# the usernames provided for the invalid login
df['user'].unique()

array(['support', 'test', 'admin', 'ftpuser', 'default', '0', '0000',
       '010101', '1111', '1234', 'api', 'dbadmin', 'ftp', 'git', 'gpadmin',
       'guest', 'monitor', 'operator', 'osmc', 'pi', 'service',
       'telecomadmin', 'ubnt', 'user', 'sybase', 'admin1', 'newadmin',
       'deployer', 'cloud', 'temp', 'dale', 'noc', 'elastic_user_7',
       'elastic_user_2', 'elastic_user_8', 'elastic_user_5',
       'elastic_user_0', 'elastic_user_6', 'elastic_user_3',
       'elastic_user_4', 'root', 'openerp', 'elastic_user_9', 'pruebas',
       'elastic_user_1', 'ajay', 'johnny', 'ems', 'bin', 'webconfig',
       'cubrid', '', 'user1'], dtype=object)

In [35]:
df.drop('timestamp',1).drop('logline',1).to_csv('C:/Users/Sotiris/Desktop/project/data/failed_auth_my_log_TEST.csv', float_format='%.f')

Folowwing lines make sure that the same results occur as desplayed by the steps illustrated in the x-pack example https://github.com/elastic/examples/blob/master/Machine%20Learning/Security%20Analytics%20Recipes/suspicious_login_activity/EXAMPLE.md

In [36]:
timestamps = df['timestamp']

In [37]:
# load datafeed extracted from the x-pack example for same dataset
import json

with open('C:/Users/Sotiris/Desktop/project/data/datafeed.json') as data_file:    
    datafeed = json.load(data_file)

In [38]:
timestamp, hostname, ip, user = [], [], [], []

for result in datafeed:
    timestamp.append(result['@timestamp'])
    hostname.append(result['system.auth.hostname'])
    ip.append(result['system.auth.ssh.ip'])
    user.append(result['system.auth.user'])
    
datafeed_df = pd.DataFrame([timestamp,hostname,ip,user]).T
datafeed_df.columns = ['timestamp', 'hostname', 'ip', 'user']

In [39]:
# check that all results match 
# a naive check of the timestamps in implemented

for i in range(len(df.index)):
    t1 = datafeed_df.iloc[i]['timestamp']

    t2 = df.iloc[i]['timestamp']
    if t1 != t2:
        print('FOUND DIFFERENCE')
        print(df.iloc[i]['logline'])